# Day 2 - Part 1
---

**PROBLEM:** Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan. The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look. Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen. To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

**EXAMPLE:** suppose you have the following list:

* 1-3 a: abcde
* 1-3 b: cdefg
* 2-9 c: ccccccccc

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

**How many passwords are valid according to their policies?**

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/d2_parsed.csv')

In [3]:
df.head()

,low,high,letter,password
0,5,11,t,glhbttzvzttkdx
1,2,4,f,cfkmf
2,9,12,m,mmmmmmmmmmmmm
3,2,10,z,vghqbzbcxf
4,10,11,b,tbtbvbbnbwd


In [4]:
df['count'] = df.apply(lambda row: row['password'].count(row['letter']), axis=1)
df.head()

,low,high,letter,password,count
0,5,11,t,glhbttzvzttkdx,4
1,2,4,f,cfkmf,2
2,9,12,m,mmmmmmmmmmmmm,13
3,2,10,z,vghqbzbcxf,1
4,10,11,b,tbtbvbbnbwd,5


In [5]:
conditions = (df['count']>=df['low']) & (df['count']<=df['high'])
df['conditions'] = np.where(conditions, 1, 0)
df.head()

,low,high,letter,password,count,conditions
0,5,11,t,glhbttzvzttkdx,4,0
1,2,4,f,cfkmf,2,1
2,9,12,m,mmmmmmmmmmmmm,13,0
3,2,10,z,vghqbzbcxf,1,0
4,10,11,b,tbtbvbbnbwd,5,0


In [6]:
answer1 = df['conditions'].sum()
print(answer1)

378

# Day 2 - Part 2
---

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting. The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. **(Be careful; Toboggan Corporate Policies have no concept of "index zero"!)** Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

* 1-3 a: abcde is valid: position 1 contains a and position 3 does not.
* 1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
* 2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.

**How many passwords are valid according to the new interpretation of the policies?**

In [61]:
df2 = pd.read_csv('data/d2_parsed.csv', names=["pos1", "pos2", "letter", "password"], header=0)
df2.head()

,pos1,pos2,letter,password
0,5,11,t,glhbttzvzttkdx
1,2,4,f,cfkmf
2,9,12,m,mmmmmmmmmmmmm
3,2,10,z,vghqbzbcxf
4,10,11,b,tbtbvbbnbwd


In [62]:
# no zero-based index
df2['pos1'] = df2['pos1']-1
df2['pos2'] = df2['pos2']-1
# create column based on string index
df2['cond1'] = df2.apply(lambda row: row['password'][row['pos1']], axis=1)
df2['cond2'] = df2.apply(lambda row: row['password'][row['pos2']], axis=1)
df2.head()

,pos1,pos2,letter,password,cond1,cond2
0,4,10,t,glhbttzvzttkdx,t,t
1,1,3,f,cfkmf,f,m
2,8,11,m,mmmmmmmmmmmmm,m,m
3,1,9,z,vghqbzbcxf,g,f
4,9,10,b,tbtbvbbnbwd,w,d


In [63]:
conditions = [((df2['cond1']==df2['letter']) & (df2['cond2']==df2['letter'])),
              ((df2['cond1']==df2['letter']) | (df2['cond2']==df2['letter']))]
choices = [0,1]
df2['criteria'] = np.select(conditions, choices, default=0)

In [64]:
df2.head()

,pos1,pos2,letter,password,cond1,cond2,criteria
0,4,10,t,glhbttzvzttkdx,t,t,0
1,1,3,f,cfkmf,f,m,1
2,8,11,m,mmmmmmmmmmmmm,m,m,0
3,1,9,z,vghqbzbcxf,g,f,0
4,9,10,b,tbtbvbbnbwd,w,d,0


In [65]:
answer2 = df2['criteria'].sum()
print(answer2)

280